In [606]:
import pandas as pd
import numpy as np
import os
import variablen as var
import warnings
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn import metrics
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt

In [586]:
filename = var.dataset_name_1
csv_file     = var.original_df_1
save_dir = var.prepair_df_dir
column = 'AMT_INCOME_TOTAL'
result_dir = var.result_dir
statistics_dir = var.statistics_dir
result_file = 'result'

In [ ]:
def split_data(csv_file,filename, column, save_dir):
    df = pd.read_csv(csv_file)
    df = df.dropna() 
    y = df.loc[:, [column]].values
    #y = df.pop(column).to_numpy()
    df = df.drop(columns=[column])
    X = df.loc[: , df.columns].values
    #  X = df.to_numpy()
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
    
    X_train = pd.DataFrame(X_train, columns= df.columns)
    X_test  = pd.DataFrame(X_test,  columns= df.columns)
    y_train = pd.DataFrame(y_train, columns= [column])
    y_test  = pd.DataFrame(y_test,  columns= [column])
    
    X_train.to_csv( save_dir  + column + '/X_train_' + filename + '.csv'  ,index=False,sep=';', decimal=',')
    X_test.to_csv(  save_dir  + column + '/X_test_'  + filename + '.csv'  ,index=False,sep=';', decimal=',')
    y_train.to_csv( save_dir  + column + '/y_train_' + filename + '.csv'  ,index=False,sep=';', decimal=',')
    y_test.to_csv(  save_dir  + column + '/y_test_'  + filename + '.csv'  ,index=False,sep=';', decimal=',')

In [ ]:
split_data(csv_file,filename, column)

In [ ]:
# Quelle https://rmisstastic.netlify.app/how-to/python/generate_html/how%20to%20generate%20missing%20values
def ampute_mcar(X_complete, missing_rate):
    # Mask completly at random some values
    M = np.random.binomial(1, missing_rate, size = X_complete.shape)
    X_obs = X_complete.copy()
    np.putmask(X_obs, M, np.nan)
    print('Percentage of newly generated mising values: {}'.\
      #Change Code:     
      format(np.round(np.sum(pd.isna(X_obs))/X_obs.size,3)))
    
    # warning if a full row is missing
    for row in X_obs:
        if np.all(pd.isna(row)):
            warnings.warn('Some row(s) contains only nan values.')
            break

    # warning if a full col is missing
    for col in X_obs.T:
        if np.all(pd.isna(col)):
            warnings.warn('Some col(s) contains only nan values.')
            break
            
    return X_obs

In [ ]:
def ampute(column_set):
    for column in column_set:
        for s in var.test_set: 
            X_complete = pd.read_csv(save_dir + column + '/' + s + '.csv' , delimiter=";", decimal=",")
            for missing_rate in var.missing_rats:
                X_obs_mcar =  pd.DataFrame(ampute_mcar(X_complete.to_numpy(), missing_rate))
                X_obs_mcar.columns = [X_complete.columns]
                filename = str(np.int_(100 * missing_rate))
                X_obs_mcar.to_csv(save_dir + column + '/' + s + '/' + filename + ".csv" ,index=False,sep=';', decimal=',')

In [ ]:
ampute(var.application_record)

In [ ]:
def means_impute(column_set):

    for column in column_set:
        X_train = pd.read_csv(save_dir + column + '/' + var.x_train + '.csv' , delimiter=";", decimal=",")
        y_train = pd.read_csv(save_dir + column + '/' + var.y_train + '.csv' , delimiter=";", decimal=",")
        y_test =  pd.read_csv(save_dir + column + '/' + var.y_test  + '.csv' , delimiter=";", decimal=",")
        
        wert = y_train[column].mean()
        y_files = os.listdir(save_dir + column + '/' + var.y_test + '/' )
        results = y_test[[column]].copy()
        for i in y_files:
            df = pd.read_csv(save_dir + column + '/' + var.y_test + '/' + i , delimiter=";", decimal=",")
            results[i[0:-4]] = df.replace(df,wert)
        results.to_csv( result_dir  + var.methode_1 + '/' + column + '.csv'  ,index=False,sep=';', decimal=',')

In [ ]:
def mode_impute(column_set):

    for column in column_set:
        X_train = pd.read_csv(save_dir + column + '/' + var.x_train + '.csv' , delimiter=";", decimal=",")
        y_train = pd.read_csv(save_dir + column + '/' + var.y_train + '.csv' , delimiter=";", decimal=",")
        y_test =  pd.read_csv(save_dir + column + '/' + var.y_test  + '.csv' , delimiter=";", decimal=",")
        
        wert = y_train[column].mode().iloc[0,]
        y_files = os.listdir(save_dir + column + '/' + var.y_test + '/' )
        results = y_test[[column]].copy()
        for i in y_files:
            df = pd.read_csv(save_dir + column + '/' + var.y_test + '/' + i , delimiter=";", decimal=",")
            results[i[0:-4]] = df.replace(df,wert)
        results.to_csv( result_dir  + var.methode_2 + '/' + column + '.csv'  ,index=False,sep=';', decimal=',')

In [ ]:
def median_impute(column_set):

    for column in column_set:
        X_train = pd.read_csv(save_dir + column + '/' + var.x_train + '.csv' , delimiter=";", decimal=",")
        y_train = pd.read_csv(save_dir + column + '/' + var.y_train + '.csv' , delimiter=";", decimal=",")
        y_test =  pd.read_csv(save_dir + column + '/' + var.y_test  + '.csv' , delimiter=";", decimal=",")
        
        wert = y_train[column].median()
        y_files = os.listdir(save_dir + column + '/' + var.y_test + '/' )
        results = y_test[[column]].copy()
        for i in y_files:
            df = pd.read_csv(save_dir + column + '/' + var.y_test + '/' + i , delimiter=";", decimal=",")
            results[i[0:-4]] = df.replace(df,wert)
        results.to_csv( result_dir  + var.methode_3 + '/' + column + '.csv'  ,index=False,sep=';', decimal=',')

In [623]:
def transform(df,trans_df):
    for column in df.columns:
        for i, v in trans_df.iterrows():
            #print(v[column])
            df[column] = df[column].replace(v[column], i)
    df[column] = df[column].astype('float64')
    return df

In [591]:
def retransform(X_original, X_complete,col):
    # iterating the columns 
    for column in X_complete.columns:
        df = pd.read_csv(var.prepair_df_dir + col + 'columns/' + column,delimiter=";", decimal=",")
        for i, v in df.iterrows():
            X_complete[column] = X_complete[column].replace(i , v[column])
            X_complete[column] = X_complete[column].astype(X_original[column].dtypes)
    return X_complete

In [613]:
def create_trans_table(df, col):
    result = pd.DataFrame()
    for column in df.columns:
        s = df[[column]].sort_values(by=[column], ascending=True).drop_duplicates(ignore_index=True)
        #s = df[[column]]
        #s.to_csv(var.prepair_df_dir + col + '/columns/' + column  ,index=False,sep=';', decimal=',',  float_format='%.2f')
        #s = pd.read_csv(var.prepair_df_dir  + col + '/columns/' + column , delimiter=";", decimal=",")
        result[column] = s[column]
    return result

In [587]:
def correlation(dataset,column):
    print(type(column))
    print(column)
    corr = dataset.corrwith(column)
    print(corr)

In [662]:
def correlation(df,column):
    corr = df.corr().abs().unstack(level=0)
    result = corr[column].drop(index=column)
    return result.idxmax()

In [ ]:
print(corr[column].max(level=0))
    kot = corr[corr>=.9]
    plt.figure(figsize=(12,8))
    sns.heatmap(kot, cmap="Greens")

In [581]:
def get_redundant_pairs(df):
    '''Get diagonal and lower triangular pairs of correlation matrix'''
    pairs_to_drop = set()
    cols = df.columns
    for i in range(0, df.shape[1]):
        for j in range(0, i+1):
            pairs_to_drop.add((cols[i], cols[j]))
    return pairs_to_drop

In [582]:
def get_top_abs_correlations(df, n=5):
    au_corr = df.corr().abs().unstack()
    labels_to_drop = get_redundant_pairs(df)
    au_corr = au_corr.drop(labels=labels_to_drop).sort_values(ascending=False)
    return au_corr[0:n]

In [714]:
def correlation_1_impute(column_set):
    for column in column_set:
        X_train = pd.read_csv(save_dir + column + '/' + var.x_train + '.csv' , delimiter=";", decimal=",")
        y_train = pd.read_csv(save_dir + column + '/' + var.y_train + '.csv' , delimiter=";", decimal=",")
        y_test =  pd.read_csv(save_dir + column + '/' + var.y_test  + '.csv' , delimiter=";", decimal=",")
        
        df = X_train.copy()
        df[y_train.columns] = y_train
        df =  df.drop(columns=['ID'])

        trans_df = create_trans_table(df, y_train.columns[0])
        df = transform(df, trans_df )
        
        col_corr = correlation(df,y_train.columns[0])
        modi_df = df[[col_corr]] 
        modi_df[y_train.columns[0]] = df[y_train.columns[0]]
        modi_df = modi_df.groupby(col_corr)[[y_train.columns[0]]].median()
        df2 =pd.DataFrame([[np.nan, np.nan]], columns=[col_corr, y_train.columns[0]])
        modi_df = modi_df.append(df2, ignore_index=True)
        
        y_files = os.listdir(save_dir + column + '/' + var.y_test + '/' )
        results = y_test[[column]].copy()
        
        
        for i in y_files:
            df_y = pd.read_csv(save_dir + column + '/' + var.y_test + '/' + i , delimiter=";", decimal=",")
            df_x = pd.read_csv(save_dir + column + '/' + var.x_test + '/' + i , delimiter=";", decimal=",")
            
            test = df_x[[col_corr]]
            test[df_y.columns[0]] = df_y
            test = transform(test, trans_df )
            print(test.head())
            test['change'] = -1
            for i, v in modi_df.iterrows():
                m = test[col_corr] == v[col_corr] 
                df.where(m, y_test.columns[0])
            m = test['change'] == -1
            df.where(m, np.nan)
            print(results.head())
            results[i[0:-4]] = test[df_y.columns[0]]
        results.to_csv( result_dir  + var.methode_4 + '/' + column + '.csv'  ,index=False,sep=';', decimal=',')

In [715]:
correlation_1_impute(var.application_record)

C:\Users\GreenBook\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
C:\Users\GreenBook\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\GreenBook\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_i

   FLAG_OWN_CAR  AMT_INCOME_TOTAL
0             0          517500.0
1             0          225000.0
2             0          157500.0
3             1          900000.0
4             0          135000.0
   AMT_INCOME_TOTAL
0          517500.0
1          225000.0
2          157500.0
3          900000.0
4          135000.0


TypeError: 'int' object is not subscriptable

In [ ]:

        s = trans_df[col_corr]
        for i, v in s.items():
            new_df = df[df[col_corr] == i ]
            mod = new_df[y_train.columns[0]].mode().iloc[0]
            df2 =pd.DataFrame([[i, mod]], columns=list('IM'))
            modi_df = modi_df.append(df2)
        df2 =pd.DataFrame([[i, mod]], columns=list('IM'))
        y_train.columns[0]
            
            
        #Mode gibt eine Serie's wieder
        #print(new_df.head())
        if not new_df[column].mode().empty:
            new_df[column].fillna(new_df[column].mode().iloc[0], inplace=True)       
        df.update(new_df)

In [ ]:
for methode in var.methodes:
    files = os.listdir(result_dir + methode+ '/' )
    for file in files:
        org_column = file[0:-4]
        result = pd.read_csv(result_dir + methode+ '/' +  file , delimiter=";", decimal=",")
        if os.path.exists(statistics_dir + 'statistics.csv'):
            statistics = pd.read_csv(statistics_dir + 'statistics.csv' , delimiter=";", decimal=",")
        else:
            statistics = pd.DataFrame()
        statistic = pd.DataFrame()
        original = result.loc[: , org_column]
        result = result.drop(columns=[org_column])
        columns = np.array(['methode','type','column', 'misssing', 'Wert'])

        
        for column in result.columns:
            values = np.array([methode,'mean_absolute_error', org_column])
            values = np.append(values, [column])
            values = np.append(values , [metrics.mean_absolute_error(original, result[column])])
            statistics = pd.DataFrame(np.array([values]), columns=columns).append(statistics, ignore_index=True)

        statistics.to_csv(statistics_dir + 'statistics.csv', index=False,sep=';', decimal=',')

In [ ]:
for methode in var.methodes:
    files = os.listdir(result_dir + methode+ '/' )
    for file in files:
        org_column = file[0:-4]
        result = pd.read_csv(result_dir + methode+ '/' +  file , delimiter=";", decimal=",")
        if os.path.exists(statistics_dir + 'statistics.csv'):
            statistics = pd.read_csv(statistics_dir + 'statistics.csv' , delimiter=";", decimal=",")

        else:
            statistics = pd.DataFrame()
            
        original = result.loc[: , org_column]
        result = result.drop(columns=[org_column])
        columns = np.array(['methode','type','column', 'misssing', 'Wert'])

        

        for column in result.columns:
            values = np.array([methode,'mean_squared_error', org_column])
            values = np.append(values, [column])
            values = np.append(values , [metrics.mean_squared_error(original, result[column])])
            statistics = pd.DataFrame(np.array([values]), columns=columns).append(statistics, ignore_index=True)

        statistics.to_csv(statistics_dir + 'statistics.csv', index=False,sep=';', decimal=',')

In [ ]:
for methode in var.methodes:
    files = os.listdir(result_dir + methode+ '/' )
    for file in files:
        org_column = file[0:-4]
        result = pd.read_csv(result_dir + methode+ '/' +  file , delimiter=";", decimal=",")
        if os.path.exists(statistics_dir + 'statistics.csv'):
            statistics = pd.read_csv(statistics_dir + 'statistics.csv' , delimiter=";", decimal=",")
        else:
            statistics = pd.DataFrame()
            
        original = result.loc[: , org_column]
        result = result.drop(columns=[org_column])
        columns = np.array(['methode','type','column', 'misssing', 'Wert'])


        for column in result.columns:
            values = np.array([methode,'root_mean_squared_error', org_column])
            values = np.append(values, [column])
            values = np.append(values , [np.sqrt(metrics.mean_squared_error(original, result[column]))])
            statistics = pd.DataFrame(np.array([values]), columns=columns).append(statistics, ignore_index=True)
        

        statistics.to_csv(statistics_dir + 'statistics.csv', index=False,sep=';', decimal=',')